In [3]:
from dslmodel import DSLModel, init_instant
from pydantic import Field
from typing import List, Optional, Any
import dspy

In [6]:
class CriteriaModel(DSLModel):
    """A model representing a criterion in the decision process."""
    name: str = Field(..., description="The name of the criterion (e.g., cost, speed, quality).")
    weight: float = Field(..., description="The weight or importance of this criterion in the decision-making process.")
    threshold: float = Field(..., description="The minimum acceptable value for this criterion (satisficing threshold).")


class OptionModel(DSLModel):
    """A model representing an available option in the decision process."""
    name: str = Field(..., description="The name of the option (e.g., option A, solution B).")
    values: dict = Field(..., description="A dictionary mapping each criterion to its performance (e.g., {'cost': 50, 'quality': 80}).")


class SatisficeModel(DSLModel):
    """A model that encapsulates the satisficing decision-making process."""
    criteria: List[CriteriaModel] = Field(..., description="A list of criteria involved in the decision-making process.")
    options: List[OptionModel] = Field(..., description="A list of available options being considered.")
    satisficing_threshold: float = Field(..., description="Overall threshold for considering a decision as satisfactory.")
    
    def evaluate(self) -> List[OptionModel]:
        """
        Evaluate all options based on satisficing criteria.
        
        Returns a list of options that meet or exceed the satisficing thresholds.
        """
        satisfice_results = []
        
        for option in self.options:
            meets_criteria = True
            for criterion in self.criteria:
                if option.values.get(criterion.name, 0) < criterion.threshold:
                    meets_criteria = False
                    break
            if meets_criteria:
                satisfice_results.append(option)
        
        return satisfice_results

    def explain(self) -> str:
        """Provide an explanation of the satisficing decision and why options were accepted or rejected."""
        evaluation = self.evaluate()

        pred = dspy.Predict("satisfice_model, evaluation -> explanation")
        return pred(satisfice_model=str(self), evaluation=str(evaluation))


In [7]:

# Example usage:

criteria_list = [
    CriteriaModel(name="cost", weight=0.5, threshold=100),
    CriteriaModel(name="quality", weight=0.3, threshold=75),
    CriteriaModel(name="speed", weight=0.2, threshold=50)
]

option_list = [
    OptionModel(name="Option A", values={"cost": 90, "quality": 80, "speed": 55}),
    OptionModel(name="Option B", values={"cost": 110, "quality": 85, "speed": 60}),
    OptionModel(name="Option C", values={"cost": 105, "quality": 70, "speed": 45})
]

init_instant()
satisfice = SatisficeModel(criteria=criteria_list, options=option_list, satisficing_threshold=50)
satisfice_results = satisfice.evaluate()
print(satisfice.explain())
